In [1]:
import re

import nltk
import pymorphy2
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer


from natasha import LocationExtractor

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

stopword_ru = []

stopword_ru = stopwords.words('russian')
stopword_ru += stopwords.words('english')
stopword_ru.append('м')
stopword_ru.append('это')
stopword_ru.append('ru')
stopword_ru.append('com')
stopword_ru.append('net')
stopword_ru.append('www')
stopword_ru.append('заголовок')
stopword_ru.append('пустой')
stopword_ru.append('главный')
stopword_ru.append('магазин')
stopword_ru.append('интернет')
stopword_ru.append('сайт')
stopword_ru.append('сайтов')
stopword_ru.append('купить')
stopword_ru.append('страница')
stopword_ru.append('онлайн')
stopword_ru.append('веб')
stopword_ru.append('ваш')
stopword_ru.append('ру')
stopword_ru.append('главный')
stopword_ru.append('класс')
stopword_ru.append('люкс')
stopword_ru.append('премиум')
stopword_ru.append('санкт')
stopword_ru.append('официальный')
stopword_ru.append('портал')
stopword_ru.append('салон')
stopword_ru.append('скидка')
stopword_ru.append('скачать')
stopword_ru.append('компания')
stopword_ru.append('клуб')
stopword_ru.append('доставка')
stopword_ru.append('центр')
stopword_ru.append('государственный')
stopword_ru.append('ооо')
stopword_ru.append('товар')
stopword_ru.append('hd')
stopword_ru.append('самый')
stopword_ru.append('продажа')
stopword_ru.append('новости')
stopword_ru.append('услуга')
stopword_ru.append('акция')
stopword_ru.append('система')
stopword_ru.append('регистрация')
stopword_ru.append('добро')
stopword_ru.append('пожаловать')
stopword_ru.append('сеть')
stopword_ru.append('метро')
stopword_ru.append('цена')
stopword_ru.append('бесплатный')
stopword_ru.append('бесплатно')
stopword_ru.append('средство')
stopword_ru.append('хороший')
stopword_ru.append('мир')
stopword_ru.append('выбор')
stopword_ru.append('pro')
stopword_ru.append('спб')
stopword_ru.append('весь')
stopword_ru.append('заказ')

names_and_cities = [x[:-1].lower() for x in list(open("Names_and_cities.txt","r"))]

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mbabaeva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mbabaeva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
cache = {}
expired = {'domain', 'expired'}

extractor_loc = LocationExtractor()

def clean_text(text):
    '''
        очистка текста от мусора
    '''
    text = re.sub(r'18\+', 'порно', text)
    text = re.sub('-\s\r\n\|-\s\r\n|\r\n|[«»]|[""]|[><]"[\[]]|//"', '', str(text))
    text = re.sub('[«»]|[""]|[><]"[\[]]"', '', text)
    text = re.sub('[0-9]|[-.,:;_%©?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|"|–|—|', ' ', text)
    text = re.sub(r'\r\n\t|\n|\\s|\r\t|\\n', ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text)
    text = re.sub('ё', 'е', text.strip().lower())
    
    '''
        очистка текста от названий городов
    '''  
    """
    for match in reversed(extractor_loc(text)):
        start, stop = match.span
        text = text.replace(text[start:stop], '')
"""
    return text

def token_stop_pymorph(text):
    '''
        [0] токенизация предложения
        [1] проверка есть ли в начале слова '-'
        [2] проверка на стоп-слова
        [3] проверка есть ли данное слово в кэше
        [4] лемматизация слова
    на выходе лист токенов
    '''

    if not isinstance(text, str):
        text = str(text)
    if text.find('domain has expired') != -1:
        return 0
    words = word_tokenize(text)
    
    words_lem = []
    for w in words:
        if w[0] == '-':
            w = w[1:]
        if len(w) == 1:
            continue
        if w in expired:
            continue
        w = morph.parse(w)[0].normal_form
        if (w not in stopword_ru) and (w not in names_and_cities):
            if w in cache:
                words_lem.append(cache[w])
            else:
                temp_cach = cache[w] = w
                words_lem.append(temp_cach)
    
    return words_lem

In [49]:
data = pd.read_csv('sites.csv')
data = data.loc[pd.notnull(data.url) & pd.notnull(data.text)].reset_index(drop=True)
data = data.loc[(data.text.apply(len) > 15) & (data.text.apply(len) < 150)].reset_index(drop=True)

In [50]:
data.head(10)

,url,text
0,http://forma1998.ru,Сеть магазинов Форма 1998 - сувениры и подарки...
1,http://azumihair.com,Cредство для восстановления волос AZUMI
2,http://lamiland.ru,ЛамиЛэнд - интересный магазин напольных покрытий.
3,http://glira.ru,Купить септик Топас в Москве - от официального...
4,http://money2mobile.ru,Пополнение телефона или электронного кошелька ...
5,http://jandi.ru,jandi.ru domain has expired
6,http://rgtvi.ru/,Информационно развлекательный Сайт
7,http://mamatov.com,Клуб здоровья и долголетия Алексея Маматова |
8,http://tyuningavto.ru,Перетяжка салона автомобиля в ателье- АвтоКомфорт
9,http://delta-parts.ru,Интернет каталог аксессуаров для автомобилей


In [7]:
hostings = ['Домен не прилинкован ни к одной из директорий на сервере!',
            'Этот домен припаркован компанией Timeweb',
            'Данный веб-сайт выставлен на продажу!',
            'masterhost - профессиональный хостинг сайта',
            'Интернет Хостинг Центра']

for hosting in hostings:
    data = data[data['text'].apply(lambda x: True if hosting not in x else False)]

In [8]:
%time

text = [clean_text(s) for s in data.text.tolist()]
text = [token_stop_pymorph(s) for s in text]

Wall time: 0 ns


C:\Users\mbabaeva\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Possible nested set at position 36
  del sys.path[0]


In [9]:
data = list((txt, url, raw_text) for txt, url, raw_text 
            in zip(text, data.url.tolist(), data.text.tolist()) if txt != 0)

In [10]:
data[:2]

[(['форма', 'сувенир', 'подарок', 'особый', 'назначение'],
  'http://forma1998.ru',
  'Сеть магазинов Форма 1998 - сувениры и подарки особого назначения'),
 (['cредство', 'восстановление', 'волос', 'azumi'],
  'http://azumihair.com',
  'Cредство для восстановления волос AZUMI')]

In [12]:
len(data)

231682

In [13]:
new_data = pd.DataFrame()

text, url, raw_text = zip(*data)
new_data['raw_text'] = raw_text
new_data['text'] = text
new_data['url'] = url

In [14]:
new_data.head(15)

,raw_text,text,url
0,Сеть магазинов Форма 1998 - сувениры и подарки...,"[форма, сувенир, подарок, особый, назначение]",http://forma1998.ru
1,Cредство для восстановления волос AZUMI,"[cредство, восстановление, волос, azumi]",http://azumihair.com
2,ЛамиЛэнд - интересный магазин напольных покрытий.,"[ламилэнд, интересный, напольный, покрытие]",http://lamiland.ru
3,Купить септик Топас в Москве - от официального...,"[септик, топас, дилер, производитель]",http://glira.ru
4,Пополнение телефона или электронного кошелька ...,"[пополнение, телефон, электронный, кошелёк, ск...",http://money2mobile.ru
5,Информационно развлекательный Сайт,"[информационно, развлекательный]",http://rgtvi.ru/
6,Клуб здоровья и долголетия Алексея Маматова |,"[здоровье, долголетие, маматовый]",http://mamatov.com
7,Перетяжка салона автомобиля в ателье- АвтоКомфорт,"[перетяжка, автомобиль, ателье, автокомфорт]",http://tyuningavto.ru
8,Интернет каталог аксессуаров для автомобилей,"[каталог, аксессуар, автомобиль]",http://delta-parts.ru
9,"Фитолампы для растений магазин в СПб, фитоламп...","[фитолампа, растение, фитолампа, фитосветильник]",http://letsgrow.ru


In [15]:
new_data = new_data.loc[new_data.text.apply(len) >= 4].reset_index(drop=True)

In [16]:
new_data.head(15)

,raw_text,text,url
0,Сеть магазинов Форма 1998 - сувениры и подарки...,"[форма, сувенир, подарок, особый, назначение]",http://forma1998.ru
1,Cредство для восстановления волос AZUMI,"[cредство, восстановление, волос, azumi]",http://azumihair.com
2,ЛамиЛэнд - интересный магазин напольных покрытий.,"[ламилэнд, интересный, напольный, покрытие]",http://lamiland.ru
3,Купить септик Топас в Москве - от официального...,"[септик, топас, дилер, производитель]",http://glira.ru
4,Пополнение телефона или электронного кошелька ...,"[пополнение, телефон, электронный, кошелёк, ск...",http://money2mobile.ru
5,Перетяжка салона автомобиля в ателье- АвтоКомфорт,"[перетяжка, автомобиль, ателье, автокомфорт]",http://tyuningavto.ru
6,"Фитолампы для растений магазин в СПб, фитоламп...","[фитолампа, растение, фитолампа, фитосветильник]",http://letsgrow.ru
7,продажа автозапчастей - автозапчасти из Японии...,"[автозапчасть, автозапчасть, япония, корея]",http://filtok2.ru
8,К миру по нитке. Статьи о разумной жизни в мире.,"[нитка, статья, разумный, жизнь]",http://cmiruponitke.ru
9,"Купить термос, термокружку, ланчбокс и термосу...","[термос, термокружка, ланчбокс, термосумка, ро...",http://kupit-termos.ru


In [17]:
text = new_data.text.tolist()

In [71]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

corp_dct = Dictionary(text, prune_at=15000) # fit dictionary, initialize a Dictionary
print(corp_dct)

Dictionary(21352 unique tokens: ['назначение', 'особый', 'подарок', 'сувенир', 'форма']...)


In [72]:
corp_dct.filter_n_most_frequent(1000) #Filter out the ‘remove_n’ most frequent tokens that appear in the documents.
print(corp_dct)

Dictionary(20352 unique tokens: ['назначение', 'особый', 'cредство', 'восстановление', 'септик']...)


In [73]:
corpus = [corp_dct.doc2bow(line) for line in text] # convert corpus to BoW format, mapping between words and their integer ids

In [77]:
len(corpus)

148328

In [78]:
#TF-IDF scheme is used for extracting features or important words which can be the best representative of your document. 
tfidf_model = TfidfModel(corpus, id2word=corp_dct) #http://www.tfidf.com/, # fit model
corpus = tfidf_model[corpus] #apply model to the corpus document

In [81]:
corpus

In [20]:
print('--here!--')

--here!--


In [21]:
flattened_corpus = [[word[1] for word in record] for record in corpus.corpus]

In [22]:
corpus = list()
for t, c, u, rt in zip(new_data.text.tolist(), flattened_corpus, 
                       new_data.url.tolist(), new_data.raw_text.tolist()):
    if len(c) > 0:
        corpus.append((rt, list(zip(t, c)), u))
        continue

In [23]:
for i, j in enumerate(corpus):
    print(i, j)
    if i > 15:
        break

0 ('Сеть магазинов Форма 1998 - сувениры и подарки особого назначения', [('форма', 1), ('сувенир', 1), ('подарок', 1), ('особый', 1), ('назначение', 1)], 'http://forma1998.ru')
1 ('Cредство для восстановления волос AZUMI', [('cредство', 1), ('восстановление', 1), ('волос', 1)], 'http://azumihair.com')
2 ('ЛамиЛэнд - интересный магазин напольных покрытий.', [('ламилэнд', 1), ('интересный', 1), ('напольный', 1)], 'http://lamiland.ru')
3 ('Купить септик Топас в Москве - от официального дилера производителя', [('септик', 1), ('топас', 1), ('дилер', 1), ('производитель', 1)], 'http://glira.ru')
4 ('Пополнение телефона или электронного кошелька через скретч-карты', [('пополнение', 1), ('телефон', 1), ('электронный', 1), ('кошелёк', 1), ('скретч', 1), ('карта', 1)], 'http://money2mobile.ru')
5 ('Перетяжка салона автомобиля в ателье- АвтоКомфорт', [('перетяжка', 1), ('автомобиль', 1), ('ателье', 1), ('автокомфорт', 1)], 'http://tyuningavto.ru')
6 ('Фитолампы для растений магазин в СПб, фитолам

In [24]:
embs = dict()
with open('ft_native_300_ru_wiki_lenta_lemmatize.vec', 'r', encoding='utf-8') as fasttext_embs:
    for ind, line in enumerate(fasttext_embs):
        if ind == 0:
            continue
        key, *value, _ = line.split(' ')
        embs[key] = np.asarray(value, dtype=np.float32)

In [25]:
final_corpus = list()
final_urls = list()
final_raws = list()

for raw_txt, record, url in corpus:
    words, tfidf_scores = zip(*record)
    best_indices = np.asarray(tfidf_scores).argsort()[-4:][::-1]
    best_words = np.asarray(words)[best_indices].tolist()
    final_corpus.append(best_words)
    final_urls.append(url)
    final_raws.append(raw_txt)

In [26]:
from sklearn.preprocessing import normalize

embs_corpus = list()

for record in final_corpus:
    result = np.zeros(300, dtype=np.float32)
    rec_len = len(record)
    for word in record:
        if word in embs:
            result += embs[word]        
    embs_corpus.append(result)
    
embs_corpus = normalize(np.asarray(embs_corpus)).tolist()

In [27]:
from sklearn.cluster import MiniBatchKMeans

In [28]:
model = MiniBatchKMeans(n_clusters=256, init='k-means++',
                        max_iter=1024, batch_size=256, verbose=0, 
                        compute_labels=True, random_state=1, tol=0.0, 
                        max_no_improvement=64, n_init=32, reassignment_ratio=0.04)

In [29]:
model.fit(embs_corpus)

MiniBatchKMeans(batch_size=256, compute_labels=True, init='k-means++',
        init_size=None, max_iter=1024, max_no_improvement=64,
        n_clusters=256, n_init=32, random_state=1, reassignment_ratio=0.04,
        tol=0.0, verbose=0)

In [30]:
model.inertia_

57380.76467968023

In [31]:
preds = model.predict(embs_corpus)

In [32]:
stats = pd.DataFrame()

In [33]:
stats['raw_text'] = final_raws
stats['text'] = [' '.join(x) for x in final_corpus]
stats['cluster'] = preds
stats['url'] = final_urls

In [34]:
stats.head(100)

,raw_text,text,cluster,url
0,Сеть магазинов Форма 1998 - сувениры и подарки...,назначение особый подарок сувенир,213,http://forma1998.ru
1,Cредство для восстановления волос AZUMI,волос восстановление cредство,108,http://azumihair.com
2,ЛамиЛэнд - интересный магазин напольных покрытий.,напольный интересный ламилэнд,151,http://lamiland.ru
3,Купить септик Топас в Москве - от официального...,производитель дилер топас септик,207,http://glira.ru
4,Пополнение телефона или электронного кошелька ...,карта скретч кошелёк электронный,111,http://money2mobile.ru
5,Перетяжка салона автомобиля в ателье- АвтоКомфорт,автокомфорт ателье автомобиль перетяжка,43,http://tyuningavto.ru
6,"Фитолампы для растений магазин в СПб, фитоламп...",растение фитолампа фитолампа,19,http://letsgrow.ru
7,продажа автозапчастей - автозапчасти из Японии...,автозапчасть япония автозапчасть,241,http://filtok2.ru
8,К миру по нитке. Статьи о разумной жизни в мире.,жизнь разумный статья нитка,117,http://cmiruponitke.ru
9,"Купить термос, термокружку, ланчбокс и термосу...",россия термосумка ланчбокс термокружка,138,http://kupit-termos.ru


In [35]:
themes = []

for i in range(stats['cluster'].max()+1):
    theme = list(stats[stats['cluster'] == i]['text'].apply(lambda x: x.split()))
    res = []
    for i in theme:
        for j in i:
            res.append(j)
    try:
        themes.append(pd.Series(res).value_counts().index[0] + ' ' + str(pd.Series(res).describe()[3] / pd.Series(res).describe()[0])[:4])
    except:
        themes.append('Undifined')
        
#print(list(zip(range(stats['cluster'].max()+1),themes)))

stats['main_theme'] = stats['cluster'].apply(lambda x: themes[x])

In [36]:
for key, i in enumerate(themes):
    print(key, i)

0 массаж 0.15
1 строительный 0.22
2 имя 0.28
3 рука 0.22
4 часы 0.20
5 hugedomains 0.02
6 такси 0.27
7 женский 0.22
8 детский 0.27
9 женщина 0.19
10 перевозка 0.08
11 рабочий 0.28
12 играть 0.00
13 футбол 0.05
14 развитие 0.20
15 стол 0.18
16 новость 0.09
17 новость 0.27
18 материал 0.03
19 растение 0.08
20 суши 0.31
21 ремонт 0.26
22 знак 0.04
23 православный 0.05
24 цветок 0.20
25 оптом 0.05
26 оборудование 0.23
27 двepь 0.16
28 дон 0.26
29 город 0.12
30 плитка 0.27
31 ребёнок 0.19
32 отель 0.19
33 порно 0.23
34 бизнес 0.26
35 продвижение 0.05
36 дверь 0.20
37 жк 0.14
38 одежда 0.11
39 тур 0.27
40 музыка 0.06
41 животное 0.05
42 детский 0.28
43 тюнинг 0.20
44 поставка 0.20
45 комплекс 0.03
46 домашний 0.26
47 iphone 0.07
48 vk 0.25
49 блог 0.46
50 форум 0.13
51 финский 0.30
52 дизайн 0.24
53 брус 0.22
54 отзыв 0.26
55 рыбалка 0.15
56 аренда 0.31
57 работа 0.04
58 мебель 0.35
59 изделие 0.06
60 лечение 0.12
61 авто 0.10
62 прокат 0.26
63 шкаф 0.10
64 юридический 0.10
65 объявление 0.3

In [37]:
stats.loc[stats.index == 10000]

,raw_text,text,cluster,url,main_theme
10000,Качественный ремонт квартиры в Калининграде от...,работа ремонтный вид метр,214,http://otdelka-kd.ru,ремонт 0.26


In [38]:
stats.loc[stats.cluster == 220]

,raw_text,text,cluster,url,main_theme
8411,"Газовые отопительные котлы в Екатеринбурге, га...",отопительный газовый газовый котлы,220,http://baza-kotlov.ru,газовый 0.30
11478,Сервисный центр по ремонту газовых котлов в Но...,котёл зип котлов газовый,220,http://zip-kotel.ru,газовый 0.30
17498,Магазин отопительного и газового оборудования....,колонка котёл оборудование газовый,220,http://kotelkolonka.ru,газовый 0.30
19582,Ремонт платы управления газового котла - Ремон...,ремонт котёл газовый управление,220,http://darsklimat.ru,газовый 0.30
26073,"Интернет-магазин ""Газовик"" - газтехника и запа...",газтехника котлы газовый часть,220,http://gazovik63.ru,газовый 0.30
26168,"Монтаж отопления, ремонт газовых колонок и ко...",котлов колонка газовый ремонт,220,http://heatproff.ru,газовый 0.30
36930,Интернет-магазин Техносбыт Киров. Продажа газо...,инструмент котлов газовый техносбыт,220,http://technoselling.ru,газовый 0.30
38809,Котел -ок интернет-магазин котлов в Екатеринбу...,ок газовый котёл котлов,220,http://kotel-ok.com,газовый 0.30
41122,Оборудование для отопления и водоснабжения | Г...,котлы газовый бытовой газовый,220,http://thermoset.ru,газовый 0.30
50583,"Газовая служба Ярославль, телефон газовой служ...",телефон служба газовый газовый,220,http://yarsgs.ru,газовый 0.30


In [40]:
stats.sort_values(by='cluster').to_csv('sites_stats.csv', index=False, sep=';')

In [41]:
a = stats[['cluster', 'url']]

In [42]:
a.head()

,cluster,url
0,213,http://forma1998.ru
1,108,http://azumihair.com
2,151,http://lamiland.ru
3,207,http://glira.ru
4,111,http://money2mobile.ru


In [43]:
a = a.groupby(['cluster'])['url'].apply(list)

In [44]:
a.head()

cluster
0    [http://smsnova.ru, http://maelewano.ru, https...
1    [http://stone-center.ru, https://ooosts.ru/, h...
2    [https://www.reg.ru/domain/shop/lot/avtolider-...
3    [http://fermerok.com, http://homehandmade.ru, ...
4    [http://xn----7sbbibna0c5aqo4e8e.xn--p1ai, htt...
Name: url, dtype: object

In [45]:
a.to_csv('grouped_clusters.csv', index=True)

In [46]:
"""
from sklearn.metrics import silhouette_score

silhouette_score(stats['text'], stats['cluster'], metric='euclidean')
"""

"\nfrom sklearn.metrics import silhouette_score\n\nsilhouette_score(stats['text'], stats['cluster'], metric='euclidean')\n"